In [ ]:
!pip install bert-extractive-summarizer

     |████████████████████████████████| 1.3MB 5.8MB/s 
     |████████████████████████████████| 890kB 18.8MB/s 
     |████████████████████████████████| 2.9MB 28.6MB/s 
     |████████████████████████████████| 1.1MB 36.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d418fb3eba13a85d9f62e31def04a709a90675ddfc7f5196e7bc88c2bcc1d240
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
get_corona_summary=pd.read_csv("/content/Hcq_Stance_Tweets_Cleaned.csv")

In [ ]:
import pandas as pd
from summarizer import Summarizer
model = Summarizer()

In [ ]:
df=get_corona_summary.url_content[:30]

In [ ]:
for i in df:
  result = model(i, min_length=30,max_length=300)
  summary = "".join(result)
  print(summary)  

In [ ]:
from summarizer import Summarizer
model = Summarizer()
result = model(get_corona_summary.url_content[1], min_length=30,max_length=100)
summary = "".join(result)
print(summary)

The studies, which have been fast-tracked by the government, will initially involve 15 NHS centres. Remdesivir has been considered as a potential treatment for Ebola.


In [ ]:
len(summary)

166

In [ ]:
len(get_corona_summary.url_content[1])

2904

In [ ]:
!pip install pyrouge

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-cp36-none-any.whl size=191613 sha256=7d6a97a835c1067405e904428d5064df145e8f6a0822d419344695c7693794ba
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


In [ ]:
from __future__ import division
from itertools import chain


def get_unigram_count(tokens):
    count_dict = dict()
    for t in tokens:
        if t in count_dict:
            count_dict[t] += 1
        else:
            count_dict[t] = 1

    return count_dict


class Rouge:
    beta = 1

    @staticmethod
    def my_lcs_grid(x, y):
        n = len(x)
        m = len(y)

        table = [[0 for i in range(m + 1)] for j in range(n + 1)]

        for j in range(m + 1):
            for i in range(n + 1):
                if i == 0 or j == 0:
                    cell = (0, 'e')
                elif x[i - 1] == y[j - 1]:
                    cell = (table[i - 1][j - 1][0] + 1, '\\')
                else:
                    over = table[i - 1][j][0]
                    left = table[i][j - 1][0]

                    if left < over:
                        cell = (over, '^')
                    else:
                        cell = (left, '<')

                table[i][j] = cell

        return table

    @staticmethod
    def my_lcs(x, y, mask_x):
        table = Rouge.my_lcs_grid(x, y)
        i = len(x)
        j = len(y)

        while i > 0 and j > 0:
            move = table[i][j][1]
            if move == '\\':
                mask_x[i - 1] = 1
                i -= 1
                j -= 1
            elif move == '^':
                i -= 1
            elif move == '<':
                j -= 1

        return mask_x

    @staticmethod
    def rouge_l(cand_sents, ref_sents):
        lcs_scores = 0.0
        cand_unigrams = get_unigram_count(chain(*cand_sents))
        ref_unigrams = get_unigram_count(chain(*ref_sents))
        for cand_sent in cand_sents:
            cand_token_mask = [0 for t in cand_sent]
            cand_len = len(cand_sent)
            for ref_sent in ref_sents:
                # aligns = []
                # Rouge.lcs(ref_sent, cand_sent, aligns)
                Rouge.my_lcs(cand_sent, ref_sent, cand_token_mask)

                # for i in aligns:
                #     ref_token_mask[i] = 1
            # lcs = []
            cur_lcs_score = 0.0
            for i in range(cand_len):
                if cand_token_mask[i]:
                    token = cand_sent[i]
                    if cand_unigrams[token] > 0 and ref_unigrams[token] > 0:
                        cand_unigrams[token] -= 1
                        ref_unigrams[token] -= 1
                        cur_lcs_score += 1

            lcs_scores += cur_lcs_score

        # print "lcs_scores: %d" % lcs_scores
        ref_words_count = sum(len(s) for s in ref_sents)
        # print "ref_words_count: %d" % ref_words_count
        cand_words_count = sum(len(s) for s in cand_sents)
        # print "cand_words_count: %d" % cand_words_count

        precision = lcs_scores / cand_words_count
        recall = lcs_scores / ref_words_count
        f_score = (1 + Rouge.beta ** 2) * precision * recall / (recall +
                                                                Rouge.beta ** 2 * precision + 1e-7) + 1e-6  # prevent underflow
        return precision, recall, f_score


In [ ]:
from summarizer import Summarizer
model = Summarizer()
result = model(get_corona_summary.url_content[1], min_length=30,max_length=300)
summary = "".join(result)
print(summary)

A drug that could help treat coronavirus is to be trialled on a small number of patients in England and Scotland. The studies, which have been fast-tracked by the government, will initially involve 15 NHS centres. Two studies are to be carried out in the UK - one on patients with moderate symptoms, and one on those who are in a serious condition. For about a decade we've been looking at what we call emerging viruses, looking at viruses that aren't a problem yet but could be in the future. " With no approved therapies for coronavirus infection, hopes rest on speeding up the approval process for drugs that show promise in fighting the disease US researchers have begun a trial trial to see if the malaria drug, chloroquine, will help treat coronavirus.


In [ ]:
r = Rouge()
system_generated_summary = "A drug that could help treat coronavirus is to be trialled on a small number of patients in England and Scotland. The studies, which have been fast-tracked by the government, will initially involve 15 NHS centres. Two studies are to be carried out in the UK - one on patients with moderate symptoms, and one on those who are in a serious condition. For about a decade we've been looking at what we call emerging viruses, looking at viruses that aren't a problem yet but could be in the future. With no approved therapies for coronavirus infection, hopes rest on speeding up the approval process for drugs that show promise in fighting the disease US researchers have begun a trial trial to see if the malaria drug, chloroquine, will help treat coronavirus."
manual_summmary = " The studies, which have been fast-tracked by the government, will initially involve 15 NHS centres. The drug, known as remdesivir, is manufactured by the pharmaceutical company Gilead. Two studies are to be carried out in the UK - one on patients with moderate symptoms, and one on those who are in a serious condition.  Remdesivir has been considered as a potential treatment for Ebola. With no approved therapies for coronavirus infection, hopes rest on speeding up the approval process for drugs that show promise in fighting the disease US researchers have begun a trial trial to see if the malaria drug, chloroquine, will help treat coronavirus."
[precision, recall, f_score] = r.rouge_l([system_generated_summary], [manual_summmary])
print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))

Precision is :0.7103174603174603
Recall is :0.8248847926267281
F Score is :0.7633271762912549


In [ ]:
result = model(get_corona_summary.url_content[4], min_length=30,max_length=500)
summary = "".join(result)
print(summary)

Presidential advisor on health, Dr. Anthony Nsiah-Asare, has revealed some of the medicines given to people infected by the deadly Coronmavirus (COVID-19) in Ghana. This controversy started when US President Donald Trump mentioned Chloroquine as potential remedies for coronavirus.


In [ ]:
r = Rouge()
system_generated_summary = "Presidential advisor on health, Dr. Anthony Nsiah-Asare, has revealed some of the medicines given to people infected by the deadly Coronmavirus (COVID-19) in Ghana. This controversy started when US President Donald Trump mentioned Chloroquine as potential remedies for coronavirus."
manual_summmary = "Ghana has so far recorded 195 cases with 5 deaths. Three (3) of the confirmed cases have fully recovered and reverted to negative status for COVID-19 after repeat test following recovery from symptoms. Apart from that forty-nine (49) of the confirmed cases have significantly improved with no symptoms and have been discharged from admission for home care pending repetition of laboratory tests. Christian Adu-Poku to comply with the president’s directive regarding the partial lockdown. Some parts of Accra, Kasoa and Tema, Kumasi and its environs are currently observing a partial lockdown in accordance with a directive by President Akufo-Addo."
[precision, recall, f_score] = r.rouge_l([system_generated_summary], [manual_summmary])
print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))

Precision is :0.5765124555160143
Recall is :0.250386398763524
F Score is :0.3491388888119199


In [ ]:
result = model(get_corona_summary.url_content[5], min_length=30,max_length=1000)
summary = "".join(result)
print(summary)

Doctors who have treated a combined total of over 425 confirmed or presumptive Chinese coronavirus patients with the drug hydroxychloroquine told Breitbart News they are finding the medication to be helpful. Studies are clearly required to more conclusively determine whether the hydroxychloroquine indeed played a role in patient recovery or whether those same patients would have recovered anyway without the medication. Some of the doctors interviewed for this article prescribed hydroxychloroquine along with zinc sulfate, and in many cases combined the drug with an antibiotic regimen. There are also concerns about the supply chain for hydroxychloroquine. The extra demand could cause a shortage for lupus and arthritis patients already using the medication regularly. Indeed, that is what the doctors say they have been finding. Also, doctors affiliated with other practices or with hospitals who are using the hydroxychloroquine and antibiotic combination say they are aware of positive resul

In [ ]:
r = Rouge()
system_generated_summary = "Doctors who have treated a combined total of over 425 confirmed or presumptive Chinese coronavirus patients with the drug hydroxychloroquine told Breitbart News they are finding the medication to be helpful. Studies are clearly required to more conclusively determine whether the hydroxychloroquine indeed played a role in patient recovery or whether those same patients would have recovered anyway without the medication. Some of the doctors interviewed for this article prescribed hydroxychloroquine along with zinc sulfate, and in many cases combined the drug with an antibiotic regimen. There are also concerns about the supply chain for hydroxychloroquine. The extra demand could cause a shortage for lupus and arthritis patients already using the medication regularly. Indeed, that is what the doctors say they have been finding. Also, doctors affiliated with other practices or with hospitals who are using the hydroxychloroquine and antibiotic combination say they are aware of positive results. Zelenko said that the lack of testing has been a factor, explaining that during the initial days he had to act clinically and that in recent weeks he needed to prescribe in large part based on symptoms, since he didn’t want to waste valuable time for the test results to return three days later. He said that the instant tests currently in development would be a gamechanger. He is still compiling more data sets. While some of the treated patients did not receive coronavirus tests due to limited availability and result times, Zelenko noted the Kiryas Joel area is flooded with coronavirus patients and he has seen many patients who fit the specific symptom profile or coronavirus as opposed to a cold or the flu. Dr. Rosy E. Joseph is a nephrologist in Hackensack, New Jersey, and has been practicing for 30 years. She graduated from Columbia University medical school. Like most area doctors, Dr. Joseph in recent weeks has experienced a surge in coronavirus patients. Joseph said she has not seen evidence of serious side effects from the prescribed cocktail. The oncologist asked that the name of the hospital be withheld from publication since he is not an official spokesperson for the facility. Both hospitals have been asked for official comment. Rubin said that after taking the cocktail for a few days, his father is “doing amazing,” and he personally is nearly recovered. Those experiences were echoed by other patients who detailed their cases to Breitbart News. The implication is that hydroxychloroquine, which has been prescribed for decades, is somehow deadly. The federal government awarded PPD Inc., a North Carolina pharmaceutical company, $750,000 for a one-month study on the use of “hydrochloroquine and chloroquine in patients with coronavirus disease (COVID-19), positive for SARS-COV-2 virus exposure, or pre-exposure and post-exposure prophylaxis,” BuzzFeed reported. The AP reported: The French study has attracted the most attention. Six also were given the antibiotic azithromycin."
manual_summmary = "They said they decided to act because they felt it was safe and because the coronavirus pandemic does not leave any time to wait for what could be months or even years of clinical trials.Studies are clearly required to more conclusively determine whether the hydroxychloroquine indeed played a role in patient recovery or whether those same patients would have recovered anyway without the medication. Even if hydroxychloroquine is found to be helpful, more research is needed to determine whether the drug needs to be combined with other medication, and which amounts should be used. Some of the doctors interviewed for this article prescribed hydroxychloroquine along with zinc sulfate, and in many cases combined the drug with an antibiotic regimen. Hospitals are reportedly seeking to stockpile the drug in anticipation of a flood of coronavirus patients and many pharmacies say they are experiencing a massive number of prescription requests for hydroxychloroquine sulfate. The extra demand could cause a shortage for lupus and arthritis patients already using the medication regularly. The FDA noted the medication is currently being used in several countries to treat Chinese coronavirus and said the “known and potential benefits of chloroquine phosphate and hydroxychloroquine sulfate” outweigh “the known and potential risks” of these drugs in treating COVID-19. The hydroxychloroquine sulfate is thought to serve as a key to force open internal cell channels, including those susceptible to coronavirus, allowing the zinc to penetrate and help line the intracellular walls to protect against coronavirus penetration. "
[precision, recall, f_score] = r.rouge_l([system_generated_summary], [manual_summmary])
print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))

Precision is :0.35079575596816975
Recall is :0.6498771498771498
F Score is :0.45564264266630317


In [ ]:
result = model(get_corona_summary.url_content[11], min_length=30,max_length=300)
summary = "".join(result)
print(summary)

A controversial anti-malaria drug is being trialed as a therapy for COVID-19 at the B.C. seniors’ care home at the epicentre of the province’s novel coronavirus outbreak. As a family we had the right to deny the treatment,” she said. “ However, its benefits remain unproven, and the B.C. Centre for Disease Control (BCCDC) has recommended against its use to treat the disease “based on the lack of clinically convincing outcomes and the fragility of the supply chain.” Alberta and other provinces are expected to join within a few days.


In [ ]:
r = Rouge()
system_generated_summary = "A controversial anti-malaria drug is being trialed as a therapy for COVID-19 at the B.C. seniors’ care home at the epicentre of the province’s novel coronavirus outbreak. As a family we had the right to deny the treatment,” she said. “ However, its benefits remain unproven, and the B.C. Centre for Disease Control (BCCDC) has recommended against its use to treat the disease “based on the lack of clinically convincing outcomes and the fragility of the supply chain.” Alberta and other provinces are expected to join within a few days."
manual_summmary = "Global News has confirmed that residents of the Lynn Valley Care Centre are receiving hydroxychloroquine as a part of an international study on the effectiveness of the drug. Forty-six residents and 24 staff at the facility have have tested positive for COVID-19. The drug is being administered to all residents of the virus-stricken building known as “the lodge” at the care home, with the exception of those who opted out of the trial. Trump says Chloroquine could be a ‘game changer. The drug made international headlines this month when U.S. President Donald Trump touted it as a potential breakthrough treatment for COVID-19. The drug has shown promise in some small, early trials. However, its benefits remain unproven, and the B.C. Centre for Disease Control (BCCDC) has recommended against its use to treat the disease “based on the lack of clinically convincing outcomes and the fragility of the supply chain.”"
[precision, recall, f_score] = r.rouge_l([system_generated_summary], [manual_summmary])
print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))

Precision is :0.746268656716418
Recall is :0.4352557127312296
F Score is :0.54982913215867


In [ ]:
import pandas as pd
df=pd.read_csv("new.csv")

In [ ]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
df=df[:20]

In [ ]:
temp = []
for i in df.url_content:
  result = model(i, min_length=30,max_length=300)
  summary = "".join(result)
  temp.append(summary)
  #print(summary)

In [ ]:
temp

['A drug that could help treat coronavirus is to be trialled on a small number of patients in England and Scotland. The studies, which have been fast-tracked by the government, will initially involve 15 NHS centres. Two studies are to be carried out in the UK - one on patients with moderate symptoms, and one on those who are in a serious condition. For about a decade we\'ve been looking at what we call emerging viruses, looking at viruses that aren\'t a problem yet but could be in the future. " With no approved therapies for coronavirus infection, hopes rest on speeding up the approval process for drugs that show promise in fighting the disease US researchers have begun a trial trial to see if the malaria drug, chloroquine, will help treat coronavirus.',
 'Presidential advisor on health, Dr. Anthony Nsiah-Asare, has revealed some of the medicines given to people infected by the deadly Coronmavirus (COVID-19) in Ghana. This controversy started when US President Donald Trump mentioned Ch

In [ ]:
r = Rouge()
for i in range(0,10):
  [precision, recall, f_score] = r.rouge_l([temp[i]], [df.gold_sum[i]])
  print("Summary[",i,"]")
  print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))
  print("-----------------------------------------------------------------------")

Summary[ 0 ]
Precision is :0.8575197889182058
Recall is :0.8507853403141361
F Score is :0.8541402404081386
-----------------------------------------------------------------------
Summary[ 1 ]
Precision is :0.5765124555160143
Recall is :0.25
F Score is :0.34876306759864756
-----------------------------------------------------------------------
Summary[ 2 ]
Precision is :0.38741721854304634
Recall is :0.6468058968058968
F Score is :0.4845844782257042
-----------------------------------------------------------------------
Summary[ 3 ]
Precision is :0.26175115207373273
Recall is :0.6180631120783461
F Score is :0.36775751372025045
-----------------------------------------------------------------------
Summary[ 4 ]
Precision is :0.13805970149253732
Recall is :0.7047619047619048
F Score is :0.2308902081746329
-----------------------------------------------------------------------
Summary[ 5 ]
Precision is :0.5928030303030303
Recall is :0.28768382352941174
F Score is :0.38737719362808065
-----

In [ ]:
for i in range(0,10):
    print("Original Document:",df.url_content[i])
    print("Predicted summary:", temp[i])
    [precision, recall, f_score] = r.rouge_l([temp[i]], [df.gold_sum[i]])
    print("\n")
    print("Summary[",i,"]")
    print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))
    print("-----------------------------------------------------------------------")
    #print("\n")

Original Document: A drug that could help treat coronavirus is to be trialled on a small number of patients in England and Scotland. The studies, which have been fast-tracked by the government, will initially involve 15 NHS centres. In the absence of a known treatment for the virus, a handful of experimental drugs are being tested globally. The drug, known as remdesivir, is manufactured by the pharmaceutical company Gilead. Two studies are to be carried out in the UK - one on patients with moderate symptoms, and one on those who are in a serious condition. Trials are already underway in China and the US, with the first results expected in the coming weeks. The UK trials will be based in England and Scotland and overseen by Dr Andrew Ustianowski, a consultant in infectious diseases. He's spent the past couple of weeks working full time helping treat COVID-19 patients and has seen first-hand how sick patients can become. "What we really need, and what we really want, is a specific treatm

In [ ]:
temp[3]

'What’s going on in the fifth largest economy in the world arguably points to a major collusion scandal in which the French government is helping Big Pharma to profit from the expansion of Covid-19. Respected Professor Christian Perronne talked about the theft live in one of France’s 24/7 info channels: “The central pharmacy for the hospitals announced today that they were facing a total rupture of stocks, that they were pillaged.” Raoult was part of a clinical trial that in which hydroxychloroquine and azithromycin healed 90% of Covid-19 cases if they were tested very early. ( Only contaminated individuals should be confined. Chloroquine costs one euro for ten pills. There’s not a peep about chloroquine. One of those in charge at the time was Jerome Salomon, now a scientific counselor to the Macron government. She also says it had been her understanding that the Paris mayoral election “would not take place” and that it was, ultimately, “a masquerade.” The second round was postponed in

In [ ]:
df.gold_sum

0     A drug that could help treat coronavirus is to...
1     Ghana has so far recorded 195 cases with 5 dea...
2     They said they decided to act because they fel...
3                                                   NaN
4     Global News has confirmed that residents of th...
5                                                   NaN
6     NY, NJ Doctors and Patients See Anecdotal Evid...
7     A debate may be raging inside the White House ...
8     A survey of over 6,000 doctors found that the ...
9     President Donald Trump's trade adviser Peter N...
10    While Dr. Anthony Fauci has urged caution in u...
11    On Monday, The New York Times reported that Do...
12                                                  NaN
13                                                   []
14                                                   []
15                                                   []
16                                                   []
17                                              

In [ ]:
system_generated_summary = "A controversial anti-malaria drug is being trialed as a therapy for COVID-19 at the B.C. seniors’ care home at the epicentre of the province’s novel coronavirus outbreak. As a family we had the right to deny the treatment,” she said. “ However, its benefits remain unproven, and the B.C. Centre for Disease Control (BCCDC) has recommended against its use to treat the disease “based on the lack of clinically convincing outcomes and the fragility of the supply chain.” Alberta and other provinces are expected to join within a few days."
manual_summmary = "Global News has confirmed that residents of the Lynn Valley Care Centre are receiving hydroxychloroquine as a part of an international study on the effectiveness of the drug. Forty-six residents and 24 staff at the facility have have tested positive for COVID-19. The drug is being administered to all residents of the virus-stricken building known as “the lodge” at the care home, with the exception of those who opted out of the trial. Trump says Chloroquine could be a ‘game changer. The drug made international headlines this month when U.S. President Donald Trump touted it as a potential breakthrough treatment for COVID-19. The drug has shown promise in some small, early trials. However, its benefits remain unproven, and the B.C. Centre for Disease Control (BCCDC) has recommended against its use to treat the disease “based on the lack of clinically convincing outcomes and the fragility of the supply chain.”"
[precision, recall, f_score] = r.rouge_l([system_generated_summary], [manual_summmary])
print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))

Precision is :0.746268656716418
Recall is :0.4352557127312296
F Score is :0.54982913215867


In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer
for i in range(0,10):
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(temp[i], df.gold_sum[i])
  print(scores)

{'rouge1': Score(precision=0.8582677165354331, recall=0.8134328358208955, fmeasure=0.8352490421455939), 'rouge2': Score(precision=0.8095238095238095, recall=0.7669172932330827, fmeasure=0.7876447876447876), 'rougeL': Score(precision=0.84251968503937, recall=0.7985074626865671, fmeasure=0.8199233716475096)}
{'rouge1': Score(precision=0.125, recall=0.3170731707317073, fmeasure=0.1793103448275862), 'rouge2': Score(precision=0.019417475728155338, recall=0.05, fmeasure=0.02797202797202797), 'rougeL': Score(precision=0.07692307692307693, recall=0.1951219512195122, fmeasure=0.11034482758620691)}
{'rouge1': Score(precision=0.6169354838709677, recall=0.3686746987951807, fmeasure=0.4615384615384615), 'rouge2': Score(precision=0.2550607287449393, recall=0.15217391304347827, fmeasure=0.19062027231467477), 'rougeL': Score(precision=0.3024193548387097, recall=0.18072289156626506, fmeasure=0.22624434389140272)}
{'rouge1': Score(precision=0.41935483870967744, recall=0.6914893617021277, fmeasure=0.5220

In [ ]:
for i in range(0,10):
  print("Summary[",i,"]")
  print("Original Document:",df.url_content[i])
  print("Gold Summary:", df.gold_sum[i])
  print("Predicted summary:", temp[i])
  print("\n")
  print("*********ROUGE-1***********")
  rouge_1 = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
  rouge_1_scores = rouge_1.score(temp[i], df.gold_sum[i])
  print(rouge_1_scores)
  print("\n")
  print("*********ROUGE-2***********")
  rouge_2 = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
  rouge_2_scores = rouge_2.score(temp[i], df.gold_sum[i])
  print(rouge_2_scores)
  print("\n")
  print("*********ROUGE-L***********")
  rouge_L = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
  rouge_L_scores = rouge_L.score(temp[i], df.gold_sum[i])
  print(rouge_L_scores)
  #print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))
  print("-----------------------------------------------------------------------")

Summary[ 0 ]
Original Document: A drug that could help treat coronavirus is to be trialled on a small number of patients in England and Scotland. The studies, which have been fast-tracked by the government, will initially involve 15 NHS centres. In the absence of a known treatment for the virus, a handful of experimental drugs are being tested globally. The drug, known as remdesivir, is manufactured by the pharmaceutical company Gilead. Two studies are to be carried out in the UK - one on patients with moderate symptoms, and one on those who are in a serious condition. Trials are already underway in China and the US, with the first results expected in the coming weeks. The UK trials will be based in England and Scotland and overseen by Dr Andrew Ustianowski, a consultant in infectious diseases. He's spent the past couple of weeks working full time helping treat COVID-19 patients and has seen first-hand how sick patients can become. "What we really need, and what we really want, is a sp